In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# data analysis and wrangling
import random as rnd
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

#
from sklearn.metrics import mean_squared_log_error

データの可視化

In [ ]:
train_df = pd.read_csv('../input/traintsv/train.tsv',sep='\t')
test_df = pd.read_csv('../input/testtsv/test.tsv',sep='\t')
combine = [train_df, test_df]

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_df.describe()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
#欠損値の確認
train_df.isna().sum()

In [ ]:
#欠損値の確認
test_df.isna().sum()

EDA

In [ ]:
#状態と価格の散布図
train_df.plot.scatter(x = 'price',y = 'item_condition_id')

#コンディション5だと値段低めな傾向が
#多分 1がコンディション良くて、5が悪い

In [ ]:
train_df.plot.scatter(x = 'price',y = 'item_condition_id')

In [ ]:
#nameの種類ごとの要素数
#import collections
#c = collections.Counter(train_df['name'])
#print(c)

#bundleが２０００ちょい
#まとめ売り説

#reserveが４５０ぐらい
#〇〇様専用的な

In [ ]:
#コンディションごとの価格のヒストグラム
new_train=train_df.query('price<200')#query
g = sns.FacetGrid(train_df, col = 'item_condition_id')
g.map(plt.hist,'price',bins = 100)
#コンディションが悪いと出品数が少ない?
#コンディションがいいと出品数が多い？が価格低い...

In [ ]:
g=sns.FacetGrid(new_train,col='shipping')
g.map(plt.hist, 'price',bins=30)


In [ ]:
train_df.head()

In [ ]:
#これだと同じメモリに別の変数名でアクセスしてるだけ
#train_copy=train_df

In [ ]:
#pythonはちゃんと.copy()使ってコピーしないとだめ
train_copy = train_df.copy()

In [ ]:
#Checking for NULL values in the columns
train_copy.isnull().any()

ブランド名

In [ ]:
brands = train_copy["brand_name"].value_counts()
print("No. of Unique Brand Names :", brands.size)
fig, ax = plt.subplots(1, 2, figsize = (13, 6))
# we skipped '0' index and started from 1st because 0th index has "unknown brands"
sns.barplot(brands[1:11].values, brands[1:11].index,ax = ax[0], edgecolor='k', linewidth=0.5, palette='rocket') 
ax[0].set_title("Top 10 Most Frequently Used Brand Names", fontsize = 13)
ax[0].set_xlabel("Counts", fontsize = 10)
ax[0].set_ylabel("Brand Name", fontsize = 10)

import pandas as pd
top10_brands = train_copy.groupby('brand_name', axis=0).mean()
df_expPrice = pd.DataFrame(top10_brands.sort_values('price', ascending = False)['price'][0:10].reset_index())
ax[1].set_title("Top 10 Most Costly Brands", fontsize = 13)
ax[1] = sns.barplot(x="brand_name", y="price", data=df_expPrice, edgecolor='k', linewidth=0.5, palette='PuRd')
ax[1].set_xlabel("Brand Name", fontsize = 10)
ax[1].set_ylabel("Sale Price", fontsize = 10)
ax[1].set_xticklabels(ax[1].get_xticklabels(),rotation=35)
plt.show()

$500 以上でブランド名

In [ ]:
train_copy.query('price > 500 & brand_name == "Unknown brand" & category_name == "Other/Other/Other"')

In [ ]:
#brand_name列にブランド名ごとの平均値段をくっつけて新しいカラムを作る
train_copy = train_copy.join(train_copy.groupby('brand_name')['price'].mean().astype(int), on = 'brand_name', rsuffix='_r')


In [ ]:
# Splitting category name
def split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label") 
    
# Splitting category name into: Main Category, SubCategory_1, SubCategory_2
train_copy['main_category'], train_copy['subcat_1'], train_copy['subcat_2'] = zip(*train_copy['category_name'].apply(lambda x: split_cat(x)))

In [ ]:
# Label Encoding
# 文字データを数値データに変換
# toNumeric
from sklearn import preprocessing
def toNumeric(data,to):
    if train_copy[data].dtype == type(object):
        le = preprocessing.LabelEncoder()
        train_copy[to] = le.fit_transform(train_copy[data].astype(str))
toNumeric('name','n_name')
toNumeric('category_name','n_category_name')
toNumeric('brand_name','n_brand_name')
toNumeric('main_category','n_main_category')
toNumeric('subcat_1','n_subcat_1')
toNumeric('subcat_2','n_subcat_2')
train_copy.head()

In [ ]:
train_copy.isnull().any()

In [ ]:
#欠損値を埋める
def fill_missing_data(data):
    data.category_name.fillna(value = "Other/Other/Other", inplace = True)
    data.brand_name.fillna(value = "Unknown brand", inplace = True)
    data.item_description.fillna(value = "No description", inplace = True)
    return data

train_copy = fill_missing_data(train_copy)
train_copy = train_copy.dropna()
print(np.shape(train_copy))
train_copy.head()

#変数ドロップ

In [ ]:
train_df = train_copy.drop(['name','category_name','brand_name','item_description','main_category','subcat_1','subcat_2'],axis=1)
test_df = test_df.drop(['name','category_name','brand_name','item_description'],axis=1)

In [ ]:
train_df

In [ ]:
train_df['price_r'].astype(int)

#学習させてみる

In [ ]:
train_data = train_df.sample(frac=0.7)
test_data = train_df.drop(train_data.index)

In [ ]:
train_data.shape,test_data.shape

In [ ]:
train_df

In [ ]:
X_train = train_data.drop('price',axis=1)
Y_train = train_data['price'].astype(int)
X_train.shape, Y_train.shape 

In [ ]:
X_train.head()

In [ ]:
Y_train.head()

In [ ]:
X_test.head()

In [ ]:
#ランダムフォレスト
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(test_data.drop('price',axis=1))

In [ ]:
print(Y_pred[140:200])

In [ ]:
len(Y_pred)

In [ ]:
Y_pred.shape,Y_train.shape

In [ ]:
print(X_test)

In [ ]:
rmsle = np.sqrt(mean_squared_log_error(test_data['price'],Y_pred))

print(rmsle)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

 # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session